In [3]:
from IPython.display import display
from app_widgets import AppWidgets

In [ ]:
# Widgets
app_widgets = AppWidgets("Create draft", "config.json")

# Populate dropdown lists
app_widgets.load_dropdown_lists()

# Create draft

## Select project

In [ ]:
display(app_widgets.projects_dropdown_boxes)
app_widgets.projects_dropdown_boxes.children[0].observe(app_widgets.load_results, names = 'value')

## Select results

In [ ]:
display(app_widgets.results_selector)

## Properties

In [ ]:
display(app_widgets.method_name_textbox, app_widgets.draft_type_dropdown, app_widgets.comments_textbox)

## Support files

In [ ]:
display(app_widgets.support_files_uploader)

## Save results

In [ ]:
display(app_widgets.bottom_buttons_hbox)
display(app_widgets.increase_buttons_size)
app_widgets.create_button.on_click(app_widgets.create_draft_openbis)
app_widgets.quit_button.on_click(app_widgets.close_notebook)